In [14]:
#THIS BLOCK OF CODE SETS THE CONNECTION TO THE TWITTER API AND SPECIFIES THE NAMES OF THE PEOPLE TO BE ANALYZED 

import tweepy
from datetime import datetime
from db_functions import check_if_candidate_in_db

#AUTHENTICATION
API_KEY = 't58MOk7TXxWhxNPjewyQURSML'
API_SECRET = 'PpsR4AwNrk6DIKkTMqxJSWEAec5CdhYgtyD62nUeiJ47GL4Dp5'
Access_Token='1450187659662577665-AVs5YmHwnoU8KB6lesPkd32qzr7re5'
Access_Token_Secret='JR8z8GdH5ko1aEmx0EbMG3YuXFZwCQPFn3FZCoc0sWx5e'
Bearer_Token='AAAAAAAAAAAAAAAAAAAAAEQpmwEAAAAAhxhZOy5WnVcuQF3OCVP8N6VWvGs%3DQJ5vq4xlHnD2M2aPXjKPcT5sPgpiZiNIALlprlbvmfb1ooL98P'

client=tweepy.Client(bearer_token=Bearer_Token,access_token=Access_Token,access_token_secret=Access_Token_Secret, wait_on_rate_limit=True)

#NAMES OF THE PEOPLE TO BE ANALIZED
candidates_list=[]

#CHECK IF THE SPECFIFIED CANDIDATES ARE IN THE DATABASE AND PREPARE THE ENVIROMENT WHERE ALL TWEETS WILL BE
for candidate,status in check_if_candidate_in_db("CLAUDIA SHEINBAUM PARDO","XOCHITL GALVEZ","MARCELO EBRARD CASAUBON","ANDRES MANUEL LOPEZ OBRADOR",
                "LUISA MARIA ALCALDE LUJAN","LUIS CRESCENCIO SANDOVAL","MARTI BATRES GUADARRAMA","CLARA BRUGADA MOLINA","MIGUEL TORRUCO GARZA",
                "SAMUEL GARCIA","LORENA CUELLAR CISNEROS","SANTIAGO TABOADA CORTINA","ALEJANDRO ARMENTA MIER",
                "DULCE MARIA SILVA HERNANDEZ","ANA LILIA RIVERA RIVERA","SERGIO GONZALEZ HERNANDEZ","JOSE NARRO CESPEDES","LILLY TELLEZ",
                "ANDREA CHAVEZ TREVIÑO","ADRIANA DAVILA FERNANDEZ","JANE CARLO LOZANO","HOMERO MENESES HERNANDEZ","AZALIA CORTES GARCIA",
                "MINERVA HERNANDEZ RAMOS", "ANABELL AVALOS ZEMPOALTECA").items():

     if status==True: candidates_list.append(candidate)
#candidates_list=list(map(lambda name: name.upper(), candidates_list)) this is illustrative to understand how to use lambda functions
tweets_per_candidate={ candidate:list() for candidate in candidates_list} #dict comprehension, udemy day 26
print(tweets_per_candidate)

#DEBE HABER UNA OPCION PARA CORREGIR EL NOMBRE SI LO ESCRIBISTE MAL 


{'CLAUDIA SHEINBAUM PARDO': [], 'XOCHITL GALVEZ': [], 'MARCELO EBRARD CASAUBON': [], 'ANDRES MANUEL LOPEZ OBRADOR': [], 'LUISA MARIA ALCALDE LUJAN': [], 'LUIS CRESCENCIO SANDOVAL': [], 'MARTI BATRES GUADARRAMA': [], 'CLARA BRUGADA MOLINA': [], 'MIGUEL TORRUCO GARZA': [], 'SAMUEL GARCIA': [], 'LORENA CUELLAR CISNEROS': [], 'SANTIAGO TABOADA CORTINA': [], 'ALEJANDRO ARMENTA MIER': [], 'DULCE MARIA SILVA HERNANDEZ': [], 'ANA LILIA RIVERA RIVERA': [], 'SERGIO GONZALEZ HERNANDEZ': [], 'JOSE NARRO CESPEDES': [], 'LILLY TELLEZ': [], 'ANDREA CHAVEZ TREVIÑO': [], 'ADRIANA DAVILA FERNANDEZ': [], 'JANE CARLO LOZANO': [], 'HOMERO MENESES HERNANDEZ': [], 'AZALIA CORTES GARCIA': [], 'MINERVA HERNANDEZ RAMOS': [], 'ANABELL AVALOS ZEMPOALTECA': []}


In [15]:
#THIS BLOCK OF CODE GETS RANDOM TWEETS RELATED TO THE PEOPLE THAT ARE BEING ANALIZED 

from datetime import datetime
import psycopg2,db_functions

# GET ALL TWEETS PER CANDIDATE 
TWEETS_LIMIT=100
for candidate in candidates_list:
    cand_id=db_functions.get_candidate_id(candidate) # QUERY THE DATABASE TO GET THE CANDIDATE´S ID
    random_tweets=client.search_recent_tweets(query=candidate,max_results=TWEETS_LIMIT,tweet_fields=["text","author_id","created_at","public_metrics","geo"],expansions=["geo.place_id"]) #GET ALL TWEETS OF ONE SINGLE CANDIDATE
    
    if random_tweets.data is not None:
        #GET THE MOST RELEVANT INFORMATION OF EACH TWEET
        for tweet in random_tweets.data:
            post_id=tweet.id
            author_id=tweet.author_id
            date=tweet.created_at.strftime('%Y-%m-%d')
            text=tweet.text
            likes=tweet.public_metrics["like_count"]
            tup=(post_id,cand_id,author_id,date,text,likes)
            tweets_per_candidate[candidate].append(tup)
            db_functions.upload_tweets(tweets_per_candidate,candidate)    #UPLOAD THE COLLECTED INFO TO THE DATABASE 
    else:
        print("This candidate has no tweets")   
        
print(tweets_per_candidate)


#OBSERVACIÓN: EL CODIGO DEBE INDICAR SI HAY TWEETS DISPONIBLES PARA CADA CANDIDATO
#DEBE MOSTRARSE CUANTOS TWEETS HABIAN ANTES Y DESPUES DE LA CONSULTA, SE SOSPECHA QUE ME ESTA ELIMIINANDO TWEETS


Response(data=[<Tweet id=1744087522739642564 text='@HdezEdilberto @casanovastuder TODO MEXICO CON LA DOCTORA CLAUDIA SHEINBAUM PARDO , PRÓXIMA PRESIDENTA CONSTITUCIONAL!!!'>, <Tweet id=1744085921778061588 text='@Dimitrova_Linda Claudia Sheinbaum Pardo será la próxima Presidente de México...!! Ladren Parásitos del PRIANRD https://t.co/Vb5go2R9B9'>, <Tweet id=1744085369224618324 text='Pero sobre todo, Andrés Manuel López Obrador, Claudia Sheinbaum Pardo y Carlos Salinas de Gortari están hermanados por la mafia estadounidense-israelí. Es por esto último que Andrés eligió a Claudia como su sucesora.'>, <Tweet id=1744085367324606870 text='La mafia estadounidense-israelí está detrás de Claudia Sheinbaum Pardo; la mafia israelí-alemana está detrás de Andrés Manuel López Obrador; la mafia británica-israelí está detrás de Carlos Salinas de Gortari.'>, <Tweet id=1744079505101140115 text='RT @SocCivilMoreUSA: Guardaré postura porque confío plenamente en @Claudiashein  y ella tendrá que navegar en

In [9]:
#THIS BLOCK OF CODE REGISTERS A NEW CANDIDATE TO THE DATABASE

import psycopg2,db_functions

candidate_name="JANE CARLO LOZANO"
political_party=("FIGURA PUBLICA","FIGURA PUBLICA")

# DATABASE CONNECTION
conn = psycopg2.connect(
dbname="political_analysis",
user="postgres",
password="sedena2023",
host="localhost"  # THIS DIRECTION CAN BE CHANGED TO A SPECIFIC SERVER
)



db_functions.insert_new_candidate(candidate_name.upper(),political_party,conn)
conn.close()
